In [ ]:
import tkinter as tk
from tkinter import messagebox
from PIL import Image, ImageTk
import random
import math

GRID_SIZE = 5
BUTTON_RADIUS = 30
WINDOW_WIDTH = 600
WINDOW_HEIGHT = 800  # Increased for cleaner layout

LEVEL_IMAGES = [
    r"C:\Users\Aafrren Mughal\Desktop\images\ocean.jpg",
    r"C:\Users\Aafrren Mughal\Desktop\images\mountains.jpg",
    r"C:\Users\Aafrren Mughal\Desktop\images\jungle.jpg",
    r"C:\Users\Aafrren Mughal\Desktop\images\beach.jpg",
    r"C:\Users\Aafrren Mughal\Desktop\images\forest.jpg",
    r"C:\Users\Aafrren Mughal\Desktop\images\desert.jpg",
    r"C:\Users\Aafrren Mughal\Desktop\images\ocean.jpg",
    r"C:\Users\Aafrren Mughal\Desktop\images\city.jpg",
    r"C:\Users\Aafrren Mughal\Desktop\images\sky.jpg",
    r"C:\Users\Aafrren Mughal\Desktop\images\snow.jpg",
]

class WordPuzzleGame:
    def __init__(self, root):
        self.root = root
        self.root.title("Word Puzzle - Interleaved Words")
        self.root.geometry(f"{WINDOW_WIDTH}x{WINDOW_HEIGHT}")
        self.root.resizable(True,True)

        self.bg_label = tk.Label(self.root)
        self.bg_label.place(x=0, y=0, relwidth=1, relheight=1)

        self.grid_buttons = []
        self.letter_buttons = []
        self.letter_coords = []
        self.selected_letters = []
        self.grid_data = [["" for _ in range(GRID_SIZE)] for _ in range(GRID_SIZE)]

        self.score = 0
        self.level = 1
        self.hints_used = 0
        self.max_hints = 2
        self.found_words = []

        self.level_word_bank = {
            1: ["APPLE", "MANGO", "PEACH", "BERRY", "GRAPE"],
            2: ["LION", "TIGER", "ZEBRA", "BEAR", "HORSE"],
            3: ["RIVER", "OCEAN", "DESERT", "FOREST", "MOUNTAIN"],
            4: ["PYTHON", "JAVA", "CPLUS", "HTML", "CSS"],
            5: ["PLANE", "CAR", "BICYCLE", "BUS", "TRAIN"],
            6: ["ELEPHANT", "LION", "TIGER", "CROC", "BEAR"],
            7: ["EARTH", "MOON", "MARS", "VENUS", "SATURN"],
            8: ["GOLD", "SILVER", "DIAMOND", "PLATINUM", "COPPER"],
            9: ["OCEAN", "RIVER", "LAKE", "POND", "SEA"],
            10: ["COMPUTER", "LAPTOP", "MOBILE", "DESKTOP", "SERVER"]
        }

        self.create_widgets()
        self.setup_level()

    def set_background_image(self):
        try:
            img_path = LEVEL_IMAGES[self.level - 1]
            img = Image.open(img_path)
            img = img.resize((WINDOW_WIDTH, WINDOW_HEIGHT), Image.Resampling.LANCZOS)
            self.bg_image = ImageTk.PhotoImage(img)
            self.bg_label.config(image=self.bg_image)
            self.bg_label.image = self.bg_image  # Prevent garbage collection
        except Exception as e:
            print(f"Error loading background: {e}")

    def create_widgets(self):
        header_bg = "#f0f8ff"
        control_bg = "#e6f7ff"
        
        self.title_label = tk.Label(
            self.root, text="🌟 Word Puzzle Game 🌟",
            font=("Helvetica", 26, "bold"), bg=header_bg, fg="#333"
        )
        self.title_label.place(relx=0.5, y=10, anchor="n")

        self.level_label = tk.Label(
            self.root, text="", font=("Helvetica", 16, "bold"),
            bg=control_bg, fg="#004d99"
        )
        self.level_label.place(x=30, y=60)

        self.score_label = tk.Label(
            self.root, text="Score: 0", font=("Helvetica", 16, "bold"),
            bg=control_bg, fg="#004d99"
        )
        self.score_label.place(x=WINDOW_WIDTH - 160, y=60)

        self.selected_label = tk.Label(
            self.root, text="", font=("Helvetica", 18), bg="white",
            fg="#333", relief="ridge", width=25, height=2
        )
        self.selected_label.place(relx=0.5, y=580, anchor="center")

        self.submit_button = tk.Button(
            self.root, text="✅ Submit", command=self.submit_word,
            font=("Helvetica", 14, "bold"), bg="#4CAF50", fg="white",
            width=12, height=1, relief="raised", bd=3
        )
        self.submit_button.place(x=150, y=720)

        self.hint_button = tk.Button(
            self.root, text="💡 Hint", command=self.show_hint,
            font=("Helvetica", 14, "bold"), bg="#2196F3", fg="white",
            width=12, height=1, relief="raised", bd=3
        )
        self.hint_button.place(x=330, y=720)

        self.create_grid()

    def create_grid(self):
        x_start = 120
        y_start = 110
        self.grid_buttons.clear()
        for i in range(GRID_SIZE):
            row = []
            for j in range(GRID_SIZE):
                btn = tk.Button(
                    self.root, text="", width=4, height=2,
                    font=("Helvetica", 16, "bold"), bg="#ffffe0", fg="#333",
                    relief="groove", bd=2
                )
                btn.place(x=x_start + j * 70, y=y_start + i * 70)
                row.append(btn)
            self.grid_buttons.append(row)

    def generate_grid_with_random_letters(self):
        for i in range(GRID_SIZE):
            for j in range(GRID_SIZE):
                letter = random.choice("ABCDEFGHIJKLMNOPQRSTUVWXYZ")
                self.grid_data[i][j] = letter
                self.grid_buttons[i][j].config(text=letter, bg="lightyellow")

    def pick_words_for_level(self):
        return self.level_word_bank.get(self.level, random.sample(list(self.level_word_bank[1]), 5))

    def setup_level(self):
        self.clear_letters()
        self.hints_used = 0
        self.found_words.clear()
        self.selected_letters.clear()
        self.level_words = self.pick_words_for_level()
        self.level_label.config(text=f"Level {self.level}")
        self.set_background_image()
        self.generate_grid_with_random_letters()
        self.create_letter_buttons()

    def create_letter_buttons(self):
        all_letters = list("".join(self.level_words))
        unique_letters = sorted(set(all_letters))
        random.shuffle(unique_letters)

        center_x = WINDOW_WIDTH // 2
        center_y = WINDOW_HEIGHT - 120  # Adjusted to avoid overlap with other elements
        radius = 140  # Adjusted for better spacing

        self.letter_buttons.clear()
        self.letter_coords.clear()

        total = len(unique_letters)
        for i, letter in enumerate(unique_letters):
            angle = math.radians(i * (360 / total))
            x = center_x + radius * math.cos(angle) - BUTTON_RADIUS
            y = center_y + radius * math.sin(angle) - BUTTON_RADIUS
            btn = tk.Button(
                self.root, text=letter, font=("Helvetica", 16, "bold"),
                bg="#00bcd4", fg="white", width=3, height=1,
                relief="raised", bd=3,
                command=lambda l=letter: self.select_letter(l)
            )
            btn.place(x=x, y=y)
            self.letter_buttons.append(btn)


    def select_letter(self, letter):
        self.selected_letters.append(letter)
        current_word = "".join(self.selected_letters)
        self.selected_label.config(text=current_word)

    def submit_word(self):
        word = self.selected_label.cget("text")
        if word in self.level_words and word not in self.found_words:
            self.found_words.append(word)
            self.score += 10
            self.highlight_word_in_grid(word)
            self.update_score()
            self.clear_selection()
            if len(self.found_words) == len(self.level_words):
                messagebox.showinfo("Level Complete!", "You've found all words! Moving to next level.")
                self.level += 1
                if self.level > 10:
                    messagebox.showinfo("Congratulations!", "You've completed all levels!")
                    self.root.quit()
                else:
                    self.setup_level()
        else:
            messagebox.showerror("Try Again", f"'{word}' is not a valid word.")
            self.clear_selection()

    def update_score(self):
        self.score_label.config(text=f"Score: {self.score}")

    def show_hint(self):
        if self.hints_used < self.max_hints:
            remaining = [w for w in self.level_words if w not in self.found_words]
            if remaining:
                first_letter = random.choice(remaining)[0]
                self.hints_used += 1
                messagebox.showinfo("Hint", f"Try a word starting with: '{first_letter}'")
            else:
                messagebox.showinfo("Hint", "All words already found!")
        else:
            messagebox.showwarning("No Hints", "You’ve used all your hints for this level!")

    def clear_letters(self):
        for btn in self.letter_buttons:
            btn.destroy()

    def clear_selection(self):
        self.selected_letters.clear()
        self.selected_label.config(text="")

    def highlight_word_in_grid(self, word):
        for i in range(GRID_SIZE):
            for j in range(GRID_SIZE - len(word) + 1):
                if all(self.grid_data[i][j + k] == word[k] for k in range(len(word))):
                    for k in range(len(word)):
                        self.grid_buttons[i][j + k].config(bg="lightgreen")
                    return
        for i in range(GRID_SIZE - len(word) + 1):
            for j in range(GRID_SIZE):
                if all(self.grid_data[i + k][j] == word[k] for k in range(len(word))):
                    for k in range(len(word)):
                        self.grid_buttons[i + k][j].config(bg="lightgreen")
                    return

if __name__ == "__main__":
    root = tk.Tk()
    game = WordPuzzleGame(root)
    root.mainloop()


In [24]:
#deepseek version
import tkinter as tk
from tkinter import messagebox
from PIL import Image, ImageTk
import random
import math

GRID_SIZE = 5
BUTTON_RADIUS = 30
WINDOW_WIDTH = 250
WINDOW_HEIGHT = 350

LEVEL_IMAGES = [
    r"C:\Users\Aafrren Mughal\Desktop\images\beach.jpg",
    r"C:\Users\Aafrren Mughal\Desktop\images\mountains.jpg",
    r"C:\Users\Aafrren Mughal\Desktop\images\jungle.jpg",
    r"C:\Users\Aafrren Mughal\Desktop\images\space.jpg",
    r"C:\Users\Aafrren Mughal\Desktop\images\forest.jpg",
    r"C:\Users\Aafrren Mughal\Desktop\images\desert.jpg",
    r"C:\Users\Aafrren Mughal\Desktop\images\ocean.jpg",
    r"C:\Users\Aafrren Mughal\Desktop\images\city.jpg",
    r"C:\Users\Aafrren Mughal\Desktop\images\sky.jpg",
    r"C:\Users\Aafrren Mughal\Desktop\images\snow.jpg",
]

class WordPuzzleGame:
    def __init__(self, root):
        self.root = root
        self.root.title("Word Puzzle - Interleaved Words")
        self.root.geometry(f"{WINDOW_WIDTH}x{WINDOW_HEIGHT}")
        self.root.resizable(False, False)

        self.bg_label = tk.Label(self.root)
        self.bg_label.place(x=0, y=0, relwidth=1, relheight=1)

        self.grid_buttons = []
        self.letter_buttons = []
        self.letter_coords = []
        self.selected_letters = []
        self.grid_data = [["" for _ in range(GRID_SIZE)] for _ in range(GRID_SIZE)]

        self.score = 0
        self.level = 1
        self.hints_used = 0
        self.max_hints = 2
        self.found_words = []

        self.level_word_bank = {
            1: ["APPLE", "MANGO", "PEACH", "BERRY", "GRAPE"],
            2: ["LION", "TIGER", "ZEBRA", "BEAR", "HORSE"],
            3: ["RIVER", "OCEAN", "DESERT", "FOREST", "MOUNTAIN"],
            4: ["PYTHON", "JAVA", "CPLUS", "HTML", "CSS"],
            5: ["PLANE", "CAR", "BICYCLE", "BUS", "TRAIN"],
            6: ["ELEPHANT", "LION", "TIGER", "CROC", "BEAR"],
            7: ["EARTH", "MOON", "MARS", "VENUS", "SATURN"],
            8: ["GOLD", "SILVER", "DIAMOND", "PLATINUM", "COPPER"],
            9: ["OCEAN", "RIVER", "LAKE", "POND", "SEA"],
            10: ["COMPUTER", "LAPTOP", "MOBILE", "DESKTOP", "SERVER"]
        }

        self.create_widgets()
        self.setup_level()

    def set_background_image(self):
        try:
            img_path = LEVEL_IMAGES[self.level - 1]
            img = Image.open(img_path)
            img = img.resize((WINDOW_WIDTH, WINDOW_HEIGHT), Image.Resampling.LANCZOS)
            self.bg_image = ImageTk.PhotoImage(img)
            self.bg_label.config(image=self.bg_image)
            self.bg_label.image = self.bg_image  # Prevent garbage collection
        except Exception as e:
            print(f"Error loading background: {e}")

    def create_widgets(self):
        header_bg = "#f0f8ff"
        control_bg = "#e6f7ff"
        
        self.title_label = tk.Label(
            self.root, text="🌟 Word Puzzle Game 🌟",
            font=("Helvetica", 24, "bold"), bg=header_bg, fg="#333"
        )
        self.title_label.place(relx=0.5, y=20, anchor="n")

        self.level_label = tk.Label(
            self.root, text="", font=("Helvetica", 16, "bold"),
            bg=control_bg, fg="#004d99"
        )
        self.level_label.place(x=40, y=80)

        self.score_label = tk.Label(
            self.root, text="Score: 0", font=("Helvetica", 16, "bold"),
            bg=control_bg, fg="#004d99"
        )
        self.score_label.place(x=WINDOW_WIDTH - 180, y=80)

        self.create_grid()
        
        # Selected word display (moved lower)
        self.selected_label = tk.Label(
            self.root, text="", font=("Helvetica", 18), bg="white",
            fg="#333", relief="ridge", width=25, height=2
        )
        self.selected_label.place(relx=0.5, y=500, anchor="center")

        # Control buttons frame
        self.control_frame = tk.Frame(self.root, bg=control_bg)
        self.control_frame.place(relx=0.5, y=550, anchor="n", width=400, height=100)

        self.submit_button = tk.Button(
            self.control_frame, text="✅ Submit", command=self.submit_word,
            font=("Helvetica", 14, "bold"), bg="#4CAF50", fg="white",
            width=12, height=1, relief="raised", bd=3
        )
        self.submit_button.pack(side="left", padx=20, pady=10)

        self.hint_button = tk.Button(
            self.control_frame, text="💡 Hint", command=self.show_hint,
            font=("Helvetica", 14, "bold"), bg="#2196F3", fg="white",
            width=12, height=1, relief="raised", bd=3
        )
        self.hint_button.pack(side="right", padx=20, pady=10)

    def create_grid(self):
        x_start = 150
        y_start = 130
        self.grid_buttons.clear()
        for i in range(GRID_SIZE):
            row = []
            for j in range(GRID_SIZE):
                btn = tk.Button(
                    self.root, text="", width=4, height=2,
                    font=("Helvetica", 16, "bold"), bg="#ffffe0", fg="#333",
                    relief="groove", bd=2
                )
                btn.place(x=x_start + j * 60, y=y_start + i * 60)
                row.append(btn)
            self.grid_buttons.append(row)

    def generate_grid_with_random_letters(self):
        for i in range(GRID_SIZE):
            for j in range(GRID_SIZE):
                letter = random.choice("ABCDEFGHIJKLMNOPQRSTUVWXYZ")
                self.grid_data[i][j] = letter
                self.grid_buttons[i][j].config(text=letter, bg="lightyellow")

    def pick_words_for_level(self):
        return self.level_word_bank.get(self.level, random.sample(list(self.level_word_bank[1]), 5))

    def setup_level(self):
        self.clear_letters()
        self.hints_used = 0
        self.found_words.clear()
        self.selected_letters.clear()
        self.level_words = self.pick_words_for_level()
        self.level_label.config(text=f"Level {self.level}")
        self.set_background_image()
        self.generate_grid_with_random_letters()
        self.create_letter_buttons()

    def create_letter_buttons(self):
        all_letters = list("".join(self.level_words))
        unique_letters = sorted(set(all_letters))
        random.shuffle(unique_letters)

        center_x = WINDOW_WIDTH // 2
        center_y = 400  # Lower position for the circle
        radius = 180

        self.letter_buttons.clear()
        self.letter_coords.clear()

        total = len(unique_letters)
        for i, letter in enumerate(unique_letters):
            angle = math.radians(i * (360 / total))
            x = center_x + radius * math.cos(angle) - BUTTON_RADIUS
            y = center_y + radius * math.sin(angle) - BUTTON_RADIUS
            btn = tk.Button(
                self.root, text=letter, font=("Helvetica", 16, "bold"),
                bg="#00bcd4", fg="white", width=3, height=1,
                relief="raised", bd=3,
                command=lambda l=letter: self.select_letter(l)
            )
            btn.place(x=x, y=y)
            self.letter_buttons.append(btn)

    def select_letter(self, letter):
        self.selected_letters.append(letter)
        self.selected_label.config(text="".join(self.selected_letters))

    def submit_word(self):
        word = self.selected_label.cget("text")
        if word in self.level_words and word not in self.found_words:
            self.found_words.append(word)
            self.score += 10
            self.highlight_word_in_grid(word)
            self.update_score()
            self.clear_selection()
            if len(self.found_words) == len(self.level_words):
                messagebox.showinfo("Level Complete!", "You've found all words! Moving to next level.")
                self.level += 1
                if self.level > 10:
                    messagebox.showinfo("Congratulations!", "You've completed all levels!")
                    self.root.quit()
                else:
                    self.setup_level()
        else:
            messagebox.showerror("Try Again", f"'{word}' is not a valid word.")
            self.clear_selection()

    def update_score(self):
        self.score_label.config(text=f"Score: {self.score}")

    def show_hint(self):
        if self.hints_used < self.max_hints:
            remaining = [w for w in self.level_words if w not in self.found_words]
            if remaining:
                first_letter = random.choice(remaining)[0]
                self.hints_used += 1
                messagebox.showinfo("Hint", f"Try a word starting with: '{first_letter}'")
            else:
                messagebox.showinfo("Hint", "All words already found!")
        else:
            messagebox.showwarning("No Hints", "You've used all your hints for this level!")

    def clear_letters(self):
        for btn in self.letter_buttons:
            btn.destroy()

    def clear_selection(self):
        self.selected_letters.clear()
        self.selected_label.config(text="")

    def highlight_word_in_grid(self, word):
        for i in range(GRID_SIZE):
            for j in range(GRID_SIZE - len(word) + 1):
                if all(self.grid_data[i][j + k] == word[k] for k in range(len(word))):
                    for k in range(len(word)):
                        self.grid_buttons[i][j + k].config(bg="lightgreen")
                    return
        for i in range(GRID_SIZE - len(word) + 1):
            for j in range(GRID_SIZE):
                if all(self.grid_data[i + k][j] == word[k] for k in range(len(word))):
                    for k in range(len(word)):
                        self.grid_buttons[i + k][j].config(bg="lightgreen")
                    return

if __name__ == "__main__":
    root = tk.Tk()
    game = WordPuzzleGame(root)
    root.mainloop()